# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    Test Scheduling
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Testing functionalities
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Example for 'free' day. (Not Working)
    </h3>
</div>

In [2]:
from ortools.sat.python import cp_model


def main():
    model = cp_model.CpModel()

    # Paramètres
    employees = ["Alice", "Bob", "Charlie", "Diana"]
    activities = ["Téléphone", "Dérogation", "Réclamation"]
    days = [f"Jour {i+1}" for i in range(10)]

    n_emp = len(employees)
    n_act = len(activities)
    n_days = len(days)

    # Variables : x[e, d, a] = 1 si employé e fait activité a le jour d
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for a in range(n_act):
                x[e, d, a] = model.NewBoolVar(f"x_e{e}_d{d}_a{a}")

    # Variable de présence : works[e, d] = 1 si l’employé e travaille le jour d
    works = {}
    for e in range(n_emp):
        for d in range(n_days):
            works[e, d] = model.NewBoolVar(f"works_e{e}_d{d}")
            # works[e,d] = 1 si au moins une activité
            model.Add(sum(x[e, d, a] for a in range(n_act)) >= 1).OnlyEnforceIf(works[e, d])
            model.Add(sum(x[e, d, a] for a in range(n_act)) == 0).OnlyEnforceIf(works[e, d].Not())

    # 1️⃣ Chaque employé fait au plus une activité par jour
    for e in range(n_emp):
        for d in range(n_days):
            model.Add(sum(x[e, d, a] for a in range(n_act)) <= 1)

    # 2️⃣ Chaque employé a exactement un jour libre sur les 10 jours
    for e in range(n_emp):
        model.Add(sum(works[e, d] for d in range(n_days)) == n_days - 1)

    # 3️⃣ Chaque jour, il faut que les 3 activités soient faites
    for d in range(n_days):
        for a in range(n_act):
            model.Add(sum(x[e, d, a] for e in range(n_emp)) == 1)

    # 4️⃣ Chaque jour, 3 employés travaillent et 1 est en congé
    for d in range(n_days):
        model.Add(sum(works[e, d] for e in range(n_emp)) == n_emp - 1)

    # 5️⃣ Optionnel : équilibrer la répartition des activités
    # (chaque employé fait environ le même nombre de chaque activité)
    for e in range(n_emp):
        total_calls = sum(x[e, d, 0] for d in range(n_days))
        total_derog = sum(x[e, d, 1] for d in range(n_days))
        total_reclam = sum(x[e, d, 2] for d in range(n_days))
        # équilibre grossier
        model.Add(total_calls >= 1)
        model.Add(total_derog >= 1)
        model.Add(total_reclam >= 1)

    # Solveur
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("✅ Solution trouvée :\n")
        for d in range(n_days):
            print(f"--- {days[d]} ---")
            for e in range(n_emp):
                if solver.Value(works[e, d]) == 0:
                    print(f"  {employees[e]} : 💤 Congé")
                else:
                    for a in range(n_act):
                        if solver.Value(x[e, d, a]) == 1:
                            print(f"  {employees[e]} : {activities[a]}")
            print()
    else:
        print("❌ Aucune solution trouvée.")


if __name__ == "__main__":
    main()


❌ Aucune solution trouvée.


In [4]:
from ortools.sat.python import cp_model

def main():
    model = cp_model.CpModel()

    employees = ["Alice", "Bob", "Charlie", "Diana"]
    activities = ["Téléphone", "Dérogation", "Réclamation"]
    days = [f"Jour {i+1}" for i in range(10)]

    n_emp = len(employees)
    n_act = len(activities)
    n_days = len(days)

    # Variables
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for a in range(n_act):
                x[e, d, a] = model.NewBoolVar(f"x_e{e}_d{d}_a{a}")

    works = {}
    for e in range(n_emp):
        for d in range(n_days):
            works[e, d] = model.NewBoolVar(f"works_e{e}_d{d}")
            model.Add(sum(x[e, d, a] for a in range(n_act)) >= 1).OnlyEnforceIf(works[e, d])
            model.Add(sum(x[e, d, a] for a in range(n_act)) == 0).OnlyEnforceIf(works[e, d].Not())

    # 1️⃣ Au plus une activité par jour
    for e in range(n_emp):
        for d in range(n_days):
            model.Add(sum(x[e, d, a] for a in range(n_act)) <= 1)

    # 2️⃣ Chaque employé a exactement un jour libre
    for e in range(n_emp):
        model.Add(sum(works[e, d] for d in range(n_days)) == n_days - 1)

    # 3️⃣ Chaque jour, les 3 activités sont couvertes
    for d in range(n_days):
        for a in range(n_act):
            model.Add(sum(x[e, d, a] for e in range(n_emp)) == 1)

    # ✅ On NE force PAS "1 employé absent chaque jour"
    # (les jours de congé se répartissent naturellement)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print("✅ Solution trouvée :\n")
        for d in range(n_days):
            print(f"--- {days[d]} ---")
            for e in range(n_emp):
                if solver.Value(works[e, d]) == 0:
                    print(f"  {employees[e]} : 💤 Congé")
                else:
                    for a in range(n_act):
                        if solver.Value(x[e, d, a]) == 1:
                            print(f"  {employees[e]} : {activities[a]}")
            print()
    else:
        print("❌ Aucune solution trouvée.")


if __name__ == "__main__":
    main()


❌ Aucune solution trouvée.


###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Testing same activities for both shifts a days
    </h3>
</div>

In [21]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, employees, days, shifts, activities, tasks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._employees = employees
        self._days = days
        self._shifts = shifts
        self._activities = activities
        self._tasks = tasks
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("     " + "           ".join(self._employees))

        for d in self._days:
            st = f"{d + 1:2} → "
            st = ""
            for e in self._employees:
                for s in self._shifts:
                    is_working = False
                    for a in self._activities:
                        if self.value(self._tasks[(e, d, s, a)]):
                            st += f"{a:5} "
                            is_working = True
                    if not is_working:
                        st += "  *   "
            print(st)
            if (d + 1) % 5 == 0:
                print()
        # print()
        # self.print_employees_statistics()
        # print()
        # self.print_days_statistics()
        # print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def print_days_statistics(self):
        for d in self._days:
            for i, s in enumerate(self._shifts):
                st = f"{d + 1:2} "
                if i:
                    st += "Matin      → "
                else:
                    st += "Après-midi → "
                total = 0
                for a in self._activities:
                    nb = sum(self.value(self._tasks[(e, d, s, a)]) for e in self._employees)
                    total += nb
                    st += f"{a}:{nb} "
                print(st + f" (nb activities: {total})")

    def print_employees_statistics(self):
        for e in self._employees:
            st = f"{e} → "
            total = 0
            for a in self._activities:
                nb = sum(self.value(self._tasks[(e, d, s, a)]) for d in self._days for s in self._shifts)
                total += nb
                st += f"{a}:{nb} "
            print(st + f" (nb activities: {total})")

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 12
n_weeks = 1
n_days = 5 * n_weeks

employees = [chr(i) for i in range(65, 65 + n_employees)]
weeks = range(n_weeks)
days = range(n_days)
shifts = ["Matin", "Après-midi"]
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp"]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# tasks[(e, d, s, a)]: employee 'e' works day 'd' on shift 's' on activity 'a'.
tasks = {}
for e in employees:
    for d in days:
        for s in shifts:
            for a in activities:
                tasks[(e, d, s, a)] = model.new_bool_var(f"task_{e}_{d}_{s}_{a}")



# ------------------- #
#                     #
#     CONSTRAINTS     #
#                     #
# ------------------- #

# 1️⃣ Each employee must have exactly one activity per shift.
for e in employees:
    for d in days:
        for s in shifts:
            model.add(sum(tasks[(e, d, s, a)] for a in activities) == 1)

# 2️⃣ Minimum / Maximum number of employees per activity.
# To be defined according to how many people are available.
for d in days:
    for s in shifts:
        # Minimum number of Tél >= 5
        model.add(sum(tasks[(e, d, s, activities[0])] for e in employees) >= 5)
        # Minimum number of Rens >= 3
        model.add(sum(tasks[(e, d, s, activities[1])] for e in employees) >= 3)
        # Minimum number of Dérog >= 1
        model.add_at_least_one(tasks[(e, d, s, activities[2])] for e in employees)
        # Minimum number of Récla == 1
        model.add_exactly_one(tasks[(e, d, s, activities[3])] for e in employees)
        # model.Add(sum(tasks[(e, d, s, activities[3])] for e in employees) == 1)
        # Minimum number of Imp == 1
        model.add_exactly_one(tasks[(e, d, s, activities[4])] for e in employees)
        # Minimum number of Libre <= 1
        # model.add_at_most_one(tasks[(e, d, s, activities[5])] for e in employees)

# 7️⃣ Same activity on morning and afternoon shifts (To Study)
# penalties = []
# for e in employees:
#     for d in days:
#         # Variable binaire : 1 si les activités matin et après-midi sont différentes
#         different_activity = model.NewBoolVar(f"different_{e}_{d}")

#         # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
#         same_activity_bools = []
#         for a in activities:
#             both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
#             model.AddBoolAnd([tasks[(e, d, shifts[0], a)], tasks[(e, d, shifts[1], a)]]).OnlyEnforceIf(both_same)
#             model.AddBoolOr([tasks[(e, d, shifts[0], a)].Not(), tasks[(e, d, shifts[1], a)].Not()]).OnlyEnforceIf(both_same.Not())
#             same_activity_bools.append(both_same)

#         # Si au moins une activité est identique sur la journée, alors different_activity = 0
#         model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
#         # Sinon, different_activity = 1
#         model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

#         # On peut ajouter une pénalité si on veut minimiser les changements d'activité
#         model.Minimize(different_activity)

penalties = []
for e in employees:
    for d in days:
        # Variable binaire : 1 si les activités matin et après-midi sont différentes
        different_activity = model.NewBoolVar(f"different_{e}_{d}")

        # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
        same_activity_bools = []
        for a in activities:
            both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
            model.AddBoolAnd([tasks[e, d, shifts[0], a], tasks[e, d, shifts[1], a]]).OnlyEnforceIf(both_same)
            model.AddBoolOr([tasks[e, d, shifts[0], a].Not(), tasks[e, d, shifts[1], a].Not()]).OnlyEnforceIf(both_same.Not())
            same_activity_bools.append(both_same)

        # Si au moins une activité est identique sur la journée, alors different_activity = 0
        model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
        # Sinon, different_activity = 1
        model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)
        penalties.append(different_activity)

# On minimise le nombre total de changements d'activité sur la semaine
model.Minimize(sum(penalties))



# -----------------------------------
# Creates the solver and solve.
# -----------------------------------

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# solver.parameters.max_time_in_seconds = 10
# solver.parameters.num_search_workers = 8
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first 'n' solutions.
solution_limit = 1
solution_printer = PartialSolutionPrinter(employees, days, shifts, activities, tasks, solution_limit)
status = solver.solve(model, solution_printer)

if status == cp_model.INFEASIBLE:
    print("Pas de solution !")

else:
    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")

Solution 1
     A           B           C           D           E           F           G           H           I           J           K           L
Récla Rens  Tél   Dérog Tél   Rens  Tél   Récla Imp   Tél   Tél   Rens  Dérog Tél   Rens  Tél   Rens  Tél   Rens  Tél   Rens  Tél   Tél   Imp   
Tél   Rens  Tél   Rens  Récla Rens  Imp   Récla Tél   Dérog Rens  Tél   Tél   Rens  Dérog Tél   Rens  Tél   Rens  Tél   Rens  Tél   Tél   Imp   
Tél   Récla Tél   Rens  Tél   Dérog Tél   Rens  Imp   Rens  Rens  Tél   Récla Tél   Dérog Tél   Rens  Tél   Rens  Tél   Rens  Tél   Tél   Imp   
Tél   Dérog Tél   Rens  Dérog Tél   Tél   Rens  Rens  Tél   Tél   Rens  Récla Tél   Imp   Imp   Rens  Tél   Rens  Tél   Rens  Tél   Tél   Récla 
Tél   Récla Récla Tél   Imp   Dérog Tél   Dérog Tél   Rens  Tél   Rens  Dérog Rens  Rens  Tél   Rens  Tél   Rens  Tél   Rens  Tél   Tél   Imp   


Stop search after 1 solutions

Statistics
  - conflicts      : 70
  - branches       : 8243
  - wall time      : 0.0466243 

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Test Qwen : New interesting approach
    </h3>
</div>

In [ ]:
from ortools.sat.python import cp_model

# -------------------------------
# Paramètres
# -------------------------------
num_employees = 12
num_days = 5
num_shifts = 2  # 0 = matin, 1 = après-midi
activities = ["Téléphone", "Renseignement", "Dérogation", "Impayés", "Réclamation"]
num_activities = len(activities)

# Contraintes minimales par shift
min_staff = {
    0: 5,  # Téléphone
    1: 3,  # Renseignement
    2: 1,  # Dérogation
    3: 1,  # Impayés
    # 4 (Réclamation) : pas de minimum
}

# -------------------------------
# Modèle
# -------------------------------
model = cp_model.CpModel()

# Variables : assignment[e][d][s] = activité (entier 0-4)
assignment = {}
for e in range(num_employees):
    for d in range(num_days):
        for s in range(num_shifts):
            assignment[(e, d, s)] = model.NewIntVar(0, num_activities - 1, f'assign_e{e}_d{d}_s{s}')

# Variables booléennes : same_activity[e][d] = 1 si matin == après-midi
same_activity = {}
for e in range(num_employees):
    for d in range(num_days):
        same_activity[(e, d)] = model.NewBoolVar(f'same_e{e}_d{d}')
        # Contrainte : same_activity ⇔ assignment[e,d,0] == assignment[e,d,1]
        model.Add(assignment[(e, d, 0)] == assignment[(e, d, 1)]).OnlyEnforceIf(same_activity[(e, d)])
        model.Add(assignment[(e, d, 0)] != assignment[(e, d, 1)]).OnlyEnforceIf(same_activity[(e, d)].Not())

# -------------------------------
# Contraintes de couverture minimale (par jour ET par shift)
# -------------------------------
for d in range(num_days):
    for s in range(num_shifts):
        # Pour chaque activité avec minimum requis
        for a in min_staff:
            # Compter combien d'employés font l'activité 'a' ce jour/shift
            bools = []
            for e in range(num_employees):
                b = model.NewBoolVar(f'count_b_e{e}_d{d}_s{s}_a{a}')
                model.Add(assignment[(e, d, s)] == a).OnlyEnforceIf(b)
                model.Add(assignment[(e, d, s)] != a).OnlyEnforceIf(b.Not())
                bools.append(b)
            model.Add(sum(bools) >= min_staff[a])

# -------------------------------
# Objectif : maximiser le nombre de (employé, jour) avec même activité matin/après-midi
# -------------------------------
total_same = model.NewIntVar(0, num_employees * num_days, 'total_same')
model.Add(total_same == sum(same_activity[(e, d)] for e in range(num_employees) for d in range(num_days)))
model.Maximize(total_same)

# -------------------------------
# Résolution
# -------------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30.0
solver.parameters.num_search_workers = 8

status = solver.Solve(model)

# -------------------------------
# Affichage
# -------------------------------
if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
    print("✅ Planning trouvé !")
    print(f"Nombre total de (employé, jour) avec même activité matin/après-midi : {solver.Value(total_same)} / {num_employees * num_days}\n")
    
    for d in range(num_days):
        print(f"--- Jour {d+1} ---")
        for e in range(num_employees):
            matin = activities[solver.Value(assignment[(e, d, 0)])]
            aprem = activities[solver.Value(assignment[(e, d, 1)])]
            meme = "✓" if solver.Value(same_activity[(e, d)]) else "✗"
            print(f"Employé {e+1:2d} : Matin = {matin:15} | Après-midi = {aprem:15} | Même ? {meme}")
        print()
else:
    print("❌ Aucun planning ne satisfait les contraintes minimales.")

✅ Planning trouvé !
Nombre total de (employé, jour) avec même activité matin/après-midi : 60 / 60

--- Jour 1 ---
Employé  1 : Matin = Téléphone       | Après-midi = Téléphone       | Même ? ✓
Employé  2 : Matin = Impayés         | Après-midi = Impayés         | Même ? ✓
Employé  3 : Matin = Téléphone       | Après-midi = Téléphone       | Même ? ✓
Employé  4 : Matin = Dérogation      | Après-midi = Dérogation      | Même ? ✓
Employé  5 : Matin = Téléphone       | Après-midi = Téléphone       | Même ? ✓
Employé  6 : Matin = Réclamation     | Après-midi = Réclamation     | Même ? ✓
Employé  7 : Matin = Renseignement   | Après-midi = Renseignement   | Même ? ✓
Employé  8 : Matin = Réclamation     | Après-midi = Réclamation     | Même ? ✓
Employé  9 : Matin = Téléphone       | Après-midi = Téléphone       | Même ? ✓
Employé 10 : Matin = Renseignement   | Après-midi = Renseignement   | Même ? ✓
Employé 11 : Matin = Téléphone       | Après-midi = Téléphone       | Même ? ✓
Employé 12 : Mati

**Add days off**

In [1]:
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

# -------------------------------
# Paramètres
# -------------------------------
num_employees = 15
num_days = 5
num_shifts = 2  # 0 = matin, 1 = après-midi
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp"]#, "Libre"]
num_activities = len(activities)

# Contraintes minimales par shift
min_staff = {
    0: 5,  # Téléphone
    1: 2,  # Renseignement
    2: 1,  # Dérogation
    3: 1,  # Réclamation
    4: 1,  # Impayés
    # 5: 0,  # Libre
}

max_staff = {
    0: 7,  # Pas plus de 7 en Téléphone
    1: 5,  # Pas plus de 5 en Renseignement
    2: 2,  # Pas plus de 2 en Dérogation
    3: 1,  # Pas plus de 2 en Réclamation
    4: 1,  # Pas plus de 2 en Impayés
    # 5: 1,  # Pas plus de 1 en Libre
}

min_monday_staff = {
    0: 3,  # Téléphone
    1: 5,  # Renseignement
    2: 1,  # Dérogation
    3: 1,  # Réclamation
    4: 1,  # Impayés
    # 5: 0,  # Libre
}

max_monday_staff = {
    0: 5,  # Pas plus de 7 en Téléphone
    1: 7,  # Pas plus de 5 en Renseignement
    2: 2,  # Pas plus de 2 en Dérogation
    3: 1,  # Pas plus de 2 en Réclamation
    4: 1,  # Pas plus de 2 en Impayés
    # 5: 1,  # Pas plus de 1 en Libre
}

# -------------------------------
# Congés :
# shift: 0 = matin, 1 = après-midi

size = 10
conges = set(zip(np.random.choice(np.arange(num_employees - 2), size = size).tolist(), 
                 np.random.choice(np.arange(num_days), size = size).tolist(),
                 [0] * size))
conges.update({(14, i, 0) for i in range(num_days)})  # Employé 15 absent tous les matins
conges.update({(14, i, 1) for i in range(num_days)})  # Employé 15 absent tous les matins


# -------------------------------
# Modèle
# -------------------------------
model = cp_model.CpModel()

# Variables
assignment = {}      # activité si présent
works = {}           # booléen : travaille ce shift ?
is_tel = {}
for e in range(num_employees):
    for d in range(num_days):
        for s in range(num_shifts):
            works[(e, d, s)] = model.NewBoolVar(f'works_e{e}_d{d}_s{s}')
            assignment[(e, d, s)] = model.NewIntVar(0, num_activities - 1, f'assign_e{e}_d{d}_s{s}')
            is_tel[(e, d, s)] = model.NewBoolVar(f'is_tel_e{e}_d{d}_s{s}')
            model.Add(assignment[(e, d, s)] == 0).OnlyEnforceIf(is_tel[(e, d, s)])
            model.Add(assignment[(e, d, s)] != 0).OnlyEnforceIf(is_tel[(e, d, s)].Not())

# Congés
for e in range(num_employees):
    for d in range(num_days):
        for s in range(num_shifts):
            # Si l'employé est en congé ce shift → works = False
            if (e, d, s) in conges:
                # print(f"Employé {e} en congé jour {d} shift {s}")
                model.Add(works[(e, d, s)] == 0)
                # model.Add(assignment[(e, d, 1 - s)] > 0)
                model.Add(is_tel[(e, d, 1 - s)] == 0)
            # Sinon, il peut travailler ou non (optionnel). Si tu veux qu'il travaille toujours sauf congé, décommente :
            else:
                model.Add(works[(e, d, s)] == 1)

# -------------------------------
# Contraintes de couverture : min ET max par shift
# -------------------------------
for d in range(num_days):
    for s in range(num_shifts):
        for a in range(num_activities):
            # Déterminer min et max pour cette activité
            min_req, max_req = 0, num_employees

            if d % 5 == 0:  # Lundi
                min_req = min_monday_staff.get(a, 0)
                max_req = max_monday_staff.get(a, num_employees)
            else:
                min_req = min_staff.get(a, 0)               # 0 si pas de min
                max_req = max_staff.get(a, num_employees)   # pas de max → tout le monde

            # Si aucune contrainte, on peut sauter (optionnel)
            if min_req == 0 and max_req >= num_employees:
                continue

            # Compter les employés présents ET affectés à l'activité 'a'
            count_vars = []
            for e in range(num_employees):
                is_assigned = model.NewBoolVar(f'is_assigned_e{e}_d{d}_s{s}_a{a}')
                # is_assigned = 1 ⇔ (works[e,d,s] == 1 ET assignment[e,d,s] == a)
                model.Add(assignment[(e, d, s)] == a).OnlyEnforceIf([works[(e, d, s)], is_assigned])
                model.Add(assignment[(e, d, s)] != a).OnlyEnforceIf([works[(e, d, s)], is_assigned.Not()])
                model.Add(is_assigned == 0).OnlyEnforceIf(works[(e, d, s)].Not())
                count_vars.append(is_assigned)

            total_count = sum(count_vars)

            # Appliquer min et max
            if min_req > 0:
                model.Add(total_count >= min_req)
            if max_req < num_employees:
                model.Add(total_count <= max_req)


# -------------------------------
# Stabilité : même activité si les deux shifts sont travaillés
# -------------------------------
same_activity = {}
for e in range(num_employees):
    for d in range(num_days):
        same_activity[(e, d)] = model.NewBoolVar(f'same_e{e}_d{d}')
        # Condition : les deux shifts sont travaillés ET activités égales
        both_work = model.NewBoolVar(f'both_work_e{e}_d{d}')
        model.AddMinEquality(both_work, [works[(e, d, 0)], works[(e, d, 1)]])

        # Si les deux travaillent, alors same_activity ⇔ activités égales
        equal_act = model.NewBoolVar(f'equal_act_e{e}_d{d}')
        model.Add(assignment[(e, d, 0)] == assignment[(e, d, 1)]).OnlyEnforceIf(equal_act)
        model.Add(assignment[(e, d, 0)] != assignment[(e, d, 1)]).OnlyEnforceIf(equal_act.Not())

        # same_activity = both_work AND equal_act
        model.AddBoolAnd([both_work, equal_act]).OnlyEnforceIf(same_activity[(e, d)])
        model.AddBoolOr([both_work.Not(), equal_act.Not()]).OnlyEnforceIf(same_activity[(e, d)].Not())

# -------------------------------
# Objectif : maximiser la stabilité
# -------------------------------
total_same = model.NewIntVar(0, num_employees * num_days, 'total_same')
model.Add(total_same == sum(same_activity.values()))
model.Maximize(total_same)

# -------------------------------
# Résolution
# -------------------------------
solver = cp_model.CpSolver()
# solver.parameters.max_time_in_seconds = 30.0
# solver.parameters.num_search_workers = 8
solver.parameters.enumerate_all_solutions = True
solution_limit = 10
status = solver.Solve(model)

# -------------------------------
# Affichage
# -------------------------------
if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
    print("✅ Planning trouvé !")
    print(f"Nombre de (employé, jour) avec même activité (et présents les 2 shifts) : {solver.Value(total_same)} / {num_employees * num_days}\n")    
    print("     " + "           ".join([chr(a) for a in range(65, 65 + num_employees)]))
    for d in range(num_days):
        st = ""
        for e in range(num_employees):
            for s in range(num_shifts):
                is_working = solver.Value(works[(e, d, s)])
                if is_working:
                    a = solver.Value(assignment[(e, d, s)])
                    st += f"{activities[a]:5} "
                else:
                    st += "  *   "
        print(st)
        if (d + 1) % 5 == 0:
            print()

    # Days statistics
    if True:
        for d in range(num_days):
            for s in range(num_shifts):
                st = f"{d + 1:2},{s} → "
                total = 0
                for a in range(len(activities)):
                    nb = 0
                    for e in range(num_employees):
                        if solver.Value(works[(e, d, s)]) and solver.Value(assignment[(e, d, s)]) == a:
                            nb += 1
                    total += nb
                    st += f"{activities[a]}:{nb} "
                print(st + f" (nb activities: {total})")
        print()

    # Employees statistics
    if True:
        for e in range(num_employees):
            st = f"{chr(65 + e)} → "
            total = 0
            for a in range(len(activities)):
                nb = 0
                for d in range(num_days):
                    for s in range(num_shifts):
                        if solver.Value(works[(e, d, s)]) and solver.Value(assignment[(e, d, s)]) == a:
                            nb += 1
                total += nb
                st += f"{activities[a]:5}:{nb:2} "
            print(st + f" (nb activities: {total})")

else:
    print("❌ Impossible de satisfaire les contraintes (vérifie les congés et les minima).")

✅ Planning trouvé !
Nombre de (employé, jour) avec même activité (et présents les 2 shifts) : 62 / 75

     A           B           C           D           E           F           G           H           I           J           K           L           M           N           O
Tél   Tél   Tél   Tél   Imp   Imp   Dérog Dérog   *   Rens  Rens  Rens  Dérog Dérog Rens  Rens  Rens  Rens  Rens  Rens  Tél   Tél   Récla Récla Tél   Tél   Rens  Rens    *     *   
  *   Rens  Tél   Tél   Tél   Tél     *   Dérog Dérog Dérog Tél   Tél   Tél   Tél   Imp   Imp   Tél   Tél     *   Rens  Rens  Rens  Rens  Rens  Rens  Rens  Récla Récla   *     *   
Tél   Tél   Tél   Tél   Tél   Tél   Tél   Tél   Rens  Rens  Imp   Imp     *   Dérog Dérog Dérog Rens  Rens  Rens  Rens  Rens  Rens  Tél   Tél   Tél   Tél   Récla Récla   *     *   
Tél   Tél   Tél   Tél   Tél   Tél   Tél   Tél   Imp   Imp   Dérog Dérog   *   Rens  Rens  Rens  Rens  Rens  Tél   Tél     *   Dérog Tél   Tél   Rens  Rens  Récla Récla   *     *  